# Text-To-Speech (TTS) 
(also called speech synthesis)

# Text-to-speech datasets
(only texts)

**Some datasets for ASR can be used to train TTS model.**

## LJSpeech

single speaker reading sentences from 7 non-fiction books in English.

## Multilingual LibriSpeech

multilingual extension of the LibriSpeech dataset for developing multilingual TTS systems and exploring cross-lingual speech synthesis techniques.

## VCTK (Voice Cloning Toolkit)

audio recordings of 110 English speakers with various accents for training TTS models with varied voices and accents, enabling more natural and diverse speech synthesis.

## Libri-TTS/ LibriTTS-R

multi-speaker English corpus of approximately 585 hours of read English speech at 24kHz sampling rate.

# Pre-trained models for text-to-speech

## SpeechT5 model

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")